<a href="https://colab.research.google.com/github/aaron-sandoval/ai-gen-papers/blob/main/gen_initial_tex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scipy --quiet
!pip install tenacity --quiet
!pip install tiktoken==0.3.3 --quiet
!pip install termcolor --quiet
!pip install arxiv --quiet
!pip install pandas --quiet
!pip install PyPDF2 --quiet
!pip install tqdm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install anthropic --quiet
!pip install sentence_transformers --quiet
!pip install arxiv --quiet
!pip install tenacity --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 56.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
import os
import arxiv
import ast
import concurrent
import json
import os
import pandas as pd
import tiktoken
from csv import writer
from IPython.display import display, Markdown, Latex
from PyPDF2 import PdfReader
from scipy import spatial
from tenacity import retry, wait_random_exponential, stop_after_attempt
from tqdm import tqdm
from termcolor import colored
import anthropic
from sentence_transformers import SentenceTransformer

ANTHROPIC_MODEL = "claude-3-5-sonnet-20240620"
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
client = anthropic.Anthropic()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key:")


Enter your Anthropic API key:··········


In [ ]:
directory = './data/papers'

# Check if the directory already exists
if not os.path.exists(directory):
    # If the directory doesn't exist, create it and any necessary intermediate directories
    os.makedirs(directory)
    print(f"Directory '{directory}' created successfully.")
else:
    # If the directory already exists, print a message indicating it
    print(f"Directory '{directory}' already exists.")

Directory './data/papers' created successfully.


In [ ]:
# Set a directory to store downloaded papers
data_dir = os.path.join(os.curdir, "data", "papers")
paper_dir_filepath = "./data/arxiv_library.csv"

# Generate a blank dataframe where we can store downloaded files
df = pd.DataFrame(list())
df.to_csv(paper_dir_filepath)

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    # Generate embeddings using the SentenceTransformer model
    embedding = embedding_model.encode(text)
    return embedding

@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def get_articles(query, library='./data/arxiv_library.csv', top_k=5):
    """Fetch the top_k articles based on a user's query, sorted by relevance.
    The articles and their embeddings are stored in the library file for later retrieval.
    """
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=10,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    result_list = []
    data_dir = "/content/data/papers"  # Specify your path here

    # Ensure the directory exists
    os.makedirs(os.path.dirname(library), exist_ok=True)

    for result in client.results(search):
        result_dict = {
            "title": result.title,
            "summary": result.summary,
            "article_url": [x.href for x in result.links][0],
            "pdf_url": [x.href for x in result.links][1]
        }
        result_list.append(result_dict)

        # Get the embedding for the article title
        embedding = embedding_request(text=result.title)

        # Store references in the library file
        file_reference = [
            result.title,
            result.download_pdf(data_dir),
            embedding.tolist()  # Convert the numpy array to a list for storing in CSV
        ]

        # Write to file
        with open(library, "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(file_reference)

    return result_list

In [ ]:
# Test that the search is working
result_output = get_articles("LLM Safety")
result_output[0]

{'title': 'Controllable Text Generation for Large Language Models: A Survey',
 'summary': "In Natural Language Processing (NLP), Large Language Models (LLMs) have\ndemonstrated high text generation quality. However, in real-world applications,\nLLMs must meet increasingly complex requirements. Beyond avoiding misleading or\ninappropriate content, LLMs are also expected to cater to specific user needs,\nsuch as imitating particular writing styles or generating text with poetic\nrichness. These varied demands have driven the development of Controllable Text\nGeneration (CTG) techniques, which ensure that outputs adhere to predefined\ncontrol conditions--such as safety, sentiment, thematic consistency, and\nlinguistic style--while maintaining high standards of helpfulness, fluency, and\ndiversity.\n  This paper systematically reviews the latest advancements in CTG for LLMs,\noffering a comprehensive definition of its core concepts and clarifying the\nrequirements for control conditions an

In [ ]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100,
) -> list[str]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = embedding_request(query)

    # Check the structure of the response and extract the embedding accordingly
    if isinstance(query_embedding_response, dict) and 'data' in query_embedding_response:
        query_embedding = query_embedding_response['data'][0]['embedding']
    elif isinstance(query_embedding_response, list):
        query_embedding = query_embedding_response[0]['embedding']
    else:
        query_embedding = query_embedding_response  # Assume it's already the embedding

    strings_and_relatednesses = [
        (row["filepath"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    return [str_rel[0] for str_rel in strings_and_relatednesses[:top_n]]

In [ ]:
from anthropic import HUMAN_PROMPT, AI_PROMPT

def read_pdf(filepath):
    """Takes a filepath to a PDF and returns a string of the PDF's contents"""
    # creating a pdf reader object
    reader = PdfReader(filepath)
    pdf_text = ""
    page_number = 0
    for page in reader.pages:
        page_number += 1
        pdf_text += page.extract_text() + f"\nPage Number: {page_number}"
    return pdf_text


# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    """Returns successive n-sized chunks from provided text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j



def extract_chunk(content, template_prompt):
    """This function applies a prompt to some input content. In this case it returns a summarized chunk of text"""
    messages = [
        {
            "role": "user",
            "content": f"{template_prompt}\n\nHere's the content to summarize:\n\n{content}"
        }
    ]
    response = client.messages.create(
        model=ANTHROPIC_MODEL,
        messages=messages,
        max_tokens=1000,
        temperature=0
    )
    return response.content[0].text

def summarize_text(query):
    """This function does the following:
    - Reads in the arxiv_library.csv file in including the embeddings
    - Finds the closest file to the user's query
    - Scrapes the text out of the file and chunks it
    - Summarizes each chunk in parallel
    - Does one final summary and returns this to the user"""

    # A prompt to dictate how the recursive summarizations should approach the input paper
    summary_prompt = """Summarize this text from an academic paper. Extract any key points with reasoning. Keep any mathematical equations and tables(if any)\n\nContent:"""

    # If the library is empty (no searches have been performed yet), we perform one and download the results
    library_df = pd.read_csv(paper_dir_filepath).reset_index()
    if len(library_df) == 0:
        print("No papers searched yet, downloading first.")
        get_articles(query)
        print("Papers downloaded, continuing")
        library_df = pd.read_csv(paper_dir_filepath).reset_index()
    library_df.columns = ["title", "filepath", "embedding"]
    library_df["embedding"] = library_df["embedding"].apply(ast.literal_eval)
    strings = strings_ranked_by_relatedness(query, library_df, top_n=1)
    print("Chunking text from paper")
    pdf_text = read_pdf(strings[0])

    # Initialise tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")
    results = ""

    # Chunk up the document into 1500 token chunks
    chunks = create_chunks(pdf_text, 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print("Summarizing each chunk of text")

    # Parallel process the summaries
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=len(text_chunks)
    ) as executor:
        futures = [
            executor.submit(extract_chunk, chunk, summary_prompt)
            for chunk in text_chunks
        ]
        with tqdm(total=len(text_chunks)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(1)
        for future in futures:
            data = future.result()
            results += data

    # Final summary
    print("Summarizing into overall summary")
    response = client.messages.create(
        model=ANTHROPIC_MODEL,
        messages=[
            {
                "role": "user",
                "content": f"""Write a summary collated from this collection of key points extracted from an academic paper.
                        The summary should highlight the core argument, conclusions and evidence, and answer the user's query.
                        User query: {query}
                        The summary should be structured in bulleted lists following the headings Core Argument, Evidence, and Conclusions. Keep all the mathematical equations
                        Key points:\n{results}\nSummary:\n"""
            }
        ],
        max_tokens=1000,
        temperature=0
    )
    return response.content[0].text

In [ ]:
# Test the summarize_text function works
chat_test_response = summarize_text("LLM Safety")

Chunking text from paper
Summarizing each chunk of text


100%|██████████| 8/8 [00:13<00:00,  1.66s/it]


Summarizing into overall summary


In [ ]:
print(chat_test_response)

Based on the key points provided from the academic paper, here is a summary addressing the user query on LLM Safety:

Core Argument:
• The paper introduces DLCRec, a novel approach for managing diversity in Large Language Model (LLM)-based recommender systems to address safety concerns around homogeneity and filter bubbles.
• DLCRec enables fine-grained control over diversity in LLM recommendations through a task decomposition strategy and data augmentation techniques.

Evidence:
• DLCRec decomposes the recommendation process into three sub-tasks: genre prediction, genre filling, and item prediction.
• Data augmentation techniques are used to address scarcity of diversity-related user behavior data:
  - Injecting noise into training samples 
  - Modifying distributions of control targets
• Extensive empirical evaluation on two real-world datasets shows DLCRec outperforms baselines across multiple recommendation scenarios.
• Key equation for fine-tuning the model: 
  max Φ Σ(x,y)∈Z Σ|y|

In [ ]:
import json
from anthropic import Anthropic

client = Anthropic()

def generate_latex_paper(query, summary):
    """
    Generate a .tex file for an academic paper based on summarized research.

    :param query: String, the original research query
    :param summary: String, the summary of analyzed papers
    :return: String, the content of the .tex file
    """

    paper_structure_prompt = f"""Based on the following summary of research papers about "{query}",
    generate a new academic paper. Include a title, authors (fictional),
    abstract, and sections (including introduction, methodology, results, discussion, and conclusion).
    For each section, provide a solid description of what should be included in a research paper format.
    All the sections must be realistic albeit short and feel like a real scientific paper.
    Do not give guidelines to write the sections but write the section completely.
    It should feel like a real 2 pager research paper that could be submitted to a conference.
    Incorporate the mathematical equations of the paper in the sections. Add tables with fictional results if applicable.Keep placeholders of two images that could be added later.
    Follow the structure of the paper rigorously, do not miss out on sections.
    This should not happen: if 'title' not in paper_structure or 'authors' not in paper_structure or 'abstract' not in paper_structure or 'sections' not in paper_structure:       raise ValueError("Missing required keys in paper_structure")


    Summary of research:
    {summary}

    Respond with a JSON structure containing the paper"""

    structure_response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        messages=[{"role": "user", "content": paper_structure_prompt}],
        max_tokens=4000
    )

    # Extract JSON from the response
    paper_structure_text = structure_response.content[0].text

    try:
        paper_structure = json.loads(paper_structure_text)
    except json.JSONDecodeError as e:
        print("JSON Decode Error:", str(e))
        raise ValueError(f"Invalid JSON structure in the response: {paper_structure_text}")

    if not isinstance(paper_structure, dict):
        raise TypeError(f"Expected paper_structure to be a dict, but got {type(paper_structure)}")

    if 'title' not in paper_structure or 'authors' not in paper_structure or 'abstract' not in paper_structure or 'sections' not in paper_structure:
        raise ValueError("Missing required keys in paper_structure")

    # Generate LaTeX content
    latex_content = r"""
\documentclass[10pt,twocolumn,letterpaper]{article}
\usepackage[utf8]{inputenc}
\usepackage{amsmath}
\usepackage{graphicx}
\usepackage[margin=1in]{geometry}

\title{%s}
\author{%s}

\begin{document}

\maketitle

\begin{abstract}
%s
\end{abstract}

""" % (paper_structure['title'], r" \and ".join(paper_structure['authors']), paper_structure['abstract'])

    sections = paper_structure['sections']
    if isinstance(sections, dict):
        for section_name, section_content in sections.items():
            latex_content += r"\section{%s}" % section_name.replace("_", " ").title()
            latex_content += "\n\n"
            if isinstance(section_content, dict) and 'description' in section_content:
                latex_content += section_content['description']
            else:
                latex_content += str(section_content)
            latex_content += "\n\n"
    elif isinstance(sections, str):
        # If sections is a string, we'll need to generate content for each section
        section_names = ["Introduction", "Methodology", "Results", "Discussion", "Conclusion"]
        for section_name in section_names:
            latex_content += r"\section{%s}" % section_name
            latex_content += "\n\n"

            section_content_prompt = f"""Generate detailed content for the {section_name} section of the paper about {query}.
            The content should be in LaTeX format and be suitable for a conference paper."""

            section_response = client.messages.create(
                model="claude-3-5-sonnet-20240620",
                messages=[{"role": "user", "content": section_content_prompt}],
                max_tokens=1500
            )

            latex_content += section_response.content[0].text
            latex_content += "\n\n"

    latex_content += r"""
\bibliographystyle{plain}
\bibliography{references}

\end{document}
"""

    return latex_content

In [ ]:
# Example usage
query = "LLM Safety"
summary = summarize_text(query)  # Assuming this function exists and returns a summary



Chunking text from paper
Summarizing each chunk of text


100%|██████████| 8/8 [00:07<00:00,  1.07it/s]


Summarizing into overall summary


In [ ]:
latex_content = generate_latex_paper(query, summary)

# Write the content to a .tex file
with open("generated_paper1.tex", "w") as f:
    f.write(latex_content)

print("LaTeX file 'generated_paper1.tex' has been generated.")

LaTeX file 'generated_paper1.tex' has been generated.


In [ ]:
# prompt: read and print the .tex file

with open("generated_paper1.tex", "r") as f:
    latex_content = f.read()

print(latex_content)



\documentclass[10pt,twocolumn,letterpaper]{article}
\usepackage[utf8]{inputenc}
\usepackage{amsmath}
\usepackage{graphicx}
\usepackage[margin=1in]{geometry}

\title{DivControl: A Framework for Fine-Grained Diversity Control in LLM-Based Recommender Systems}
\author{Sophia Zhang \and Alex Patel \and Maria Rodriguez \and David Chen}

\begin{document}

\maketitle

\begin{abstract}
Large Language Model (LLM)-based recommender systems have shown promising results in various domains, but they often struggle with generating diverse recommendations, leading to potential echo chambers and filter bubbles. We present DivControl, a novel framework for managing diversity in LLM-based recommenders while maintaining recommendation accuracy. Our approach employs a task decomposition strategy and introduces data augmentation techniques to address the scarcity of diversity-related user behavior data. Experiments on two real-world datasets demonstrate that DivControl outperforms state-of-the-art baselin